# Preterm Infant Cardio-Respiratory Signals Database

Export manually selected segments

In [8]:
import os
import numpy as np
from picsdb import *

In [9]:
xls_file="./input_settings.xlsx"
segments = load_segments(xls_file, verbose=False)

Check correct loading:

In [10]:
for key_subj in segments:
    for key_seg in segments[key_subj]:
        pass
        print(key_subj, key_seg, "\n", segments[key_subj][key_seg])

infant1 segment0 
 {'on': 3550, 'off': 3750, 'freq_lo_ecg': 20.0, 'freq_hi_ecg': 120.0, 'freq_lo_resp': 0.5, 'freq_hi_resp': 5.0, 'thr_ecg': 0.2, 'thr_resp': 0.25, 'f_max_ecg': 10.0, 'f_max_resp': 2.5}
infant1 segment1 
 {'on': 4720, 'off': 4822, 'freq_lo_ecg': 20.0, 'freq_hi_ecg': 120.0, 'freq_lo_resp': 0.5, 'freq_hi_resp': 5.0, 'thr_ecg': 0.18, 'thr_resp': 0.25, 'f_max_ecg': 10.0, 'f_max_resp': 2.5}
infant1 segment2 
 {'on': 5100, 'off': 5280, 'freq_lo_ecg': 20.0, 'freq_hi_ecg': 120.0, 'freq_lo_resp': 0.5, 'freq_hi_resp': 5.0, 'thr_ecg': 0.2, 'thr_resp': 0.25, 'f_max_ecg': 10.0, 'f_max_resp': 2.5}
infant1 segment3 
 {'on': 7760, 'off': 7870, 'freq_lo_ecg': 20.0, 'freq_hi_ecg': 120.0, 'freq_lo_resp': 0.5, 'freq_hi_resp': 5.0, 'thr_ecg': 0.18, 'thr_resp': 0.25, 'f_max_ecg': 10.0, 'f_max_resp': 2.5}
infant1 segment4 
 {'on': 11435, 'off': 11550, 'freq_lo_ecg': 20.0, 'freq_hi_ecg': 120.0, 'freq_lo_resp': 0.5, 'freq_hi_resp': 5.0, 'thr_ecg': 0.18, 'thr_resp': 0.25, 'f_max_ecg': 10.0, 'f_m

## File paths

Define export folder, create if necessary:

In [11]:
base_dir = os.getcwd()
print("working directory: ", base_dir)
data_dir = os.path.join(base_dir, "./preterm-infant-cardio-respiratory-signals-database-1.0.0")
#os.chdir(data_dir)
print(f"base directory: {base_dir:s}")
print(f"data directory: {data_dir:s}")
export_folder = os.path.join(base_dir,"exports")
print(f"export folder: {export_folder:s}")
if not os.path.isdir(export_folder):
    os.mkdir(export_folder)
    print("export folder created...")
else:
    print("export folder already exists...")

working directory:  /home/frederic/Projects/picsdb
base directory: /home/frederic/Projects/picsdb
data directory: /home/frederic/Projects/picsdb/./preterm-infant-cardio-respiratory-signals-database-1.0.0
export folder: /home/frederic/Projects/picsdb/exports
export folder already exists...


## Apply settings & export

Using the manually defined settings reloaded from `input_settings.xlsx` and export the results to a new folder `exports`.  

**NOTE**: this cell contains a quick & dirty hack to invert one of the ECG signals whose R peaks pointed downwards.

In [12]:
for i_subj, key_subj in enumerate(segments):
    for i_seg, key_seg in enumerate(segments[key_subj]):
        file_index = i_subj+1
        segment_index = i_seg
        print(f"Processing: {key_subj:s} {key_seg:s}")
        print(f"file_index: {file_index:d}\nsegment_index: {segment_index:d}")
        file_ecg = f"infant{file_index:d}_ecg"
        file_resp = f"infant{file_index:d}_resp"
        print("Loading ECG file: ", file_ecg)
        print("Loading RESP file: ", file_resp)
        # data import
        x_ecg_full, x_resp_full, fs_ecg, fs_resp = load_waveforms(data_dir, file_index)
        
        # Sampling frequencies [Hz] and sampling intervals [seconds]:
        dt_ecg = 1/fs_ecg # ECG sampling interval in sec.
        dt_resp = 1/fs_resp # RESP sampling interval in sec.
        #print("ECG sampling interval dt = ", dt_ecg, " sec.")
        #print("RESP sampling interval dt = ", dt_resp, " sec.")
        
        # get segment borders in seconds from the 'segments' dictionary
        t0_sec = segments[key_subj][key_seg]['on']
        t1_sec = segments[key_subj][key_seg]['off']
        # convert seconds to samples, respect different sampling rates for ECG and RESP signals
        t0_sample_ecg = round(t0_sec * fs_ecg)
        t1_sample_ecg = round(t1_sec * fs_ecg)
        t0_sample_resp = round(t0_sec * fs_resp)
        t1_sample_resp = round(t1_sec * fs_resp)
        
        # Extract the ECG and RESP data arrays and convert them to a 1-dimensional arrays:
        x_ecg = x_ecg_full[t0_sample_ecg:t1_sample_ecg]
        x_resp = x_resp_full[t0_sample_resp:t1_sample_resp]
        
        #''' Invert ECG signal if necessary
        if (key_subj=='infant10'):
            #mn = np.min(x_ecg-np.median(x_ecg))
            #mx = np.max(x_ecg-np.median(x_ecg))
            #if np.abs(mn) > np.abs(mx):
            print("Invert ECG signal!")
            x_ecg = -x_ecg
        #'''
        
        # time axes
        time_ecg = np.arange(x_ecg.shape[0])*dt_ecg # ECG time axis
        time_resp = np.arange(x_resp.shape[0])*dt_resp # RESP time axis
        
        # cut-off frequencies for band-pass filtering
        freq_lo_ecg = segments[key_subj][key_seg]['freq_lo_ecg']
        freq_hi_ecg = segments[key_subj][key_seg]['freq_hi_ecg']
        freq_lo_resp = segments[key_subj][key_seg]['freq_lo_resp']
        freq_hi_resp = segments[key_subj][key_seg]['freq_hi_resp']
        
        # apply the and-pass filter
        x_ecg_filt  = bp_filter(x_ecg, fs_ecg, freq_lo_ecg, freq_hi_ecg)
        x_resp_filt = bp_filter(x_resp, fs_resp, freq_lo_resp, freq_hi_resp)
        
        # get thresholds
        thr_ecg = segments[key_subj][key_seg]['thr_ecg']
        thr_resp = segments[key_subj][key_seg]['thr_resp']
        # local maxima (1st round)
        locmax_ecg = locmax(x_ecg_filt)
        locmax_resp = locmax(x_resp_filt)
        w = 1
        ecg_peaks = np.array([np.mean(x_ecg_filt[i-w:i+w]) for i in locmax_ecg])
        resp_peaks = np.array([np.mean(x_resp_filt[i-w:i+w]) for i in locmax_resp])
        # apply thresholds
        locmax_ecg = np.array([j for i, j in enumerate(locmax_ecg) if ecg_peaks[i] > thr_ecg])
        locmax_resp = np.array([j for i, j in enumerate(locmax_resp) if resp_peaks[i] > thr_resp])
        
        # remove ECG peaks too close together
        f_max_ecg = segments[key_subj][key_seg]['f_max_ecg']
        pp_min_ecg = 1/f_max_ecg # sec.
        n_min_ecg = np.round(pp_min_ecg/dt_ecg)
        locmax_ecg_copy = locmax_ecg.copy()
        for i in range(len(locmax_ecg)-1):
            if (locmax_ecg[i+1]-locmax_ecg[i] < n_min_ecg):
                locmax_ecg_copy[i+1] = 0
        locmax_ecg = locmax_ecg_copy
        locmax_ecg = locmax_ecg[locmax_ecg>0]
        
        # remove RESP peaks too close together
        f_max_resp = segments[key_subj][key_seg]['f_max_resp']
        pp_min_resp = 1/f_max_resp # sec.
        n_min_resp = np.round(pp_min_resp/dt_resp)
        locmax_resp_copy = locmax_resp.copy()
        for i in range(len(locmax_resp)-1):
            if (locmax_resp[i+1]-locmax_resp[i] < n_min_resp):
                locmax_resp_copy[i+1] = 0
        locmax_resp = locmax_resp_copy
        locmax_resp = locmax_resp[locmax_resp>0]
        
        # interval statistics
        print("\nECG stats:")
        ecg_intervals = interval_stats(locmax_ecg, dt_ecg)
        print(ecg_intervals)
        
        print("\nRESP stats:")
        resp_intervals = interval_stats(locmax_resp, dt_resp)
        print(resp_intervals)
        
        # save
        if (key_subj=='infant10'):
            x_ecg = -x_ecg
            x_ecg_filt = -x_ecg_filt
            
        file_time_ecg = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg_time.txt")
        file_time_resp = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp_time.txt")
        np.savetxt(file_time_ecg, time_ecg, fmt='%.5f', delimiter='\n')
        np.savetxt(file_time_resp, time_resp, fmt='%.5f', delimiter='\n')
        
        file_ecg  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg.txt")
        file_resp = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp.txt")
        np.savetxt(file_ecg, x_ecg, fmt='%.5f', delimiter='\n')
        np.savetxt(file_resp, x_resp, fmt='%.5f', delimiter='\n')
        
        file_ecg_filt  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg_filt.txt")
        file_resp_filt = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp_filt.txt")
        np.savetxt(file_ecg_filt, x_ecg_filt, fmt='%.5f', delimiter='\n')
        np.savetxt(file_resp_filt, x_resp_filt, fmt='%.5f', delimiter='\n')
        
        file_locmax_ecg  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_locmax_ecg.txt")
        file_locmax_resp = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_locmax_resp.txt")
        np.savetxt(file_locmax_ecg, locmax_ecg, fmt='%.5f', delimiter='\n')
        np.savetxt(file_locmax_resp, locmax_resp, fmt='%.5f', delimiter='\n')
        
        file_ecg_intervals  = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_ecg_intervals.txt")
        file_resp_intervals = os.path.join(export_folder, f"{key_subj:s}_{key_seg:s}_resp_intervals.txt")
        np.savetxt(file_ecg_intervals, ecg_intervals, fmt='%.5f', delimiter='\n')
        np.savetxt(file_resp_intervals, resp_intervals, fmt='%.5f', delimiter='\n')
print("\n\n---DONE---")

Processing: infant1 segment0
file_index: 1
segment_index: 0
Loading ECG file:  infant1_ecg
Loading RESP file:  infant1_resp
Loading ECG file:  infant1_ecg
Loading RESP file:  infant1_resp
ECG sampling frequency:  250  Hz
RESP sampling frequency:  500  Hz

ECG stats:
Interval stats (n=513) :
min: 0.37 sec
max: 0.44 sec
mean: 0.39 sec (= 2.57 Hz)
std: 0.01 sec
q25: 0.38 sec
q75: 0.39 sec
[0.388 0.388 0.392 0.384 0.392 0.392 0.384 0.392 0.388 0.388 0.392 0.392
 0.388 0.392 0.388 0.392 0.388 0.384 0.388 0.384 0.384 0.384 0.384 0.38
 0.384 0.388 0.38  0.384 0.384 0.384 0.384 0.388 0.388 0.392 0.388 0.392
 0.396 0.396 0.392 0.392 0.392 0.388 0.392 0.388 0.388 0.392 0.388 0.404
 0.412 0.412 0.404 0.396 0.404 0.396 0.4   0.396 0.392 0.396 0.388 0.392
 0.388 0.388 0.384 0.388 0.384 0.384 0.384 0.384 0.384 0.38  0.384 0.384
 0.38  0.384 0.384 0.38  0.388 0.384 0.384 0.384 0.388 0.384 0.384 0.38
 0.384 0.38  0.38  0.384 0.384 0.38  0.396 0.388 0.396 0.388 0.4   0.396
 0.404 0.4   0.404 0.4   0.4 

Processing: infant1 segment3
file_index: 1
segment_index: 3
Loading ECG file:  infant1_ecg
Loading RESP file:  infant1_resp
Loading ECG file:  infant1_ecg
Loading RESP file:  infant1_resp
ECG sampling frequency:  250  Hz
RESP sampling frequency:  500  Hz

ECG stats:
Interval stats (n=269) :
min: 0.38 sec
max: 0.44 sec
mean: 0.41 sec (= 2.45 Hz)
std: 0.01 sec
q25: 0.40 sec
q75: 0.42 sec
[0.38  0.384 0.384 0.38  0.388 0.384 0.392 0.392 0.396 0.412 0.416 0.412
 0.408 0.416 0.412 0.412 0.408 0.412 0.412 0.412 0.408 0.416 0.408 0.412
 0.408 0.412 0.42  0.42  0.416 0.42  0.412 0.416 0.416 0.412 0.412 0.408
 0.42  0.416 0.42  0.424 0.428 0.432 0.424 0.424 0.416 0.416 0.408 0.408
 0.408 0.404 0.404 0.4   0.404 0.396 0.4   0.4   0.4   0.4   0.4   0.4
 0.4   0.4   0.4   0.408 0.4   0.404 0.404 0.412 0.416 0.412 0.412 0.408
 0.412 0.408 0.42  0.416 0.42  0.42  0.412 0.416 0.412 0.412 0.412 0.408
 0.404 0.404 0.404 0.4   0.4   0.4   0.396 0.396 0.396 0.396 0.396 0.392
 0.392 0.396 0.392 0.392 0.38

Processing: infant2 segment2
file_index: 2
segment_index: 2
Loading ECG file:  infant2_ecg
Loading RESP file:  infant2_resp
Loading ECG file:  infant2_ecg
Loading RESP file:  infant2_resp
ECG sampling frequency:  500  Hz
RESP sampling frequency:  50  Hz

ECG stats:
Interval stats (n=224) :
min: 0.50 sec
max: 0.61 sec
mean: 0.53 sec (= 1.88 Hz)
std: 0.02 sec
q25: 0.52 sec
q75: 0.54 sec
[0.56  0.538 0.54  0.538 0.53  0.524 0.534 0.524 0.53  0.544 0.538 0.54
 0.546 0.536 0.536 0.552 0.592 0.538 0.544 0.548 0.55  0.558 0.55  0.544
 0.538 0.546 0.54  0.546 0.536 0.532 0.532 0.552 0.544 0.524 0.522 0.518
 0.512 0.52  0.51  0.522 0.532 0.526 0.534 0.542 0.546 0.54  0.546 0.546
 0.552 0.564 0.59  0.58  0.564 0.614 0.59  0.574 0.592 0.578 0.546 0.526
 0.528 0.522 0.522 0.524 0.538 0.546 0.53  0.534 0.544 0.546 0.56  0.546
 0.536 0.524 0.516 0.516 0.522 0.526 0.538 0.542 0.542 0.532 0.534 0.532
 0.556 0.542 0.54  0.54  0.546 0.544 0.542 0.536 0.544 0.53  0.524 0.524
 0.52  0.528 0.526 0.552 0.59

Processing: infant3 segment0
file_index: 3
segment_index: 0
Loading ECG file:  infant3_ecg
Loading RESP file:  infant3_resp
Loading ECG file:  infant3_ecg
Loading RESP file:  infant3_resp
ECG sampling frequency:  500  Hz
RESP sampling frequency:  50  Hz

ECG stats:
Interval stats (n=363) :
min: 0.46 sec
max: 0.54 sec
mean: 0.51 sec (= 1.97 Hz)
std: 0.02 sec
q25: 0.50 sec
q75: 0.52 sec
[0.53  0.536 0.54  0.53  0.534 0.518 0.506 0.496 0.494 0.492 0.488 0.492
 0.496 0.5   0.494 0.506 0.5   0.494 0.508 0.502 0.514 0.512 0.5   0.51
 0.518 0.514 0.508 0.5   0.498 0.488 0.492 0.492 0.498 0.508 0.508 0.512
 0.516 0.508 0.526 0.514 0.508 0.498 0.496 0.494 0.494 0.492 0.496 0.498
 0.492 0.496 0.504 0.514 0.512 0.52  0.522 0.53  0.532 0.518 0.522 0.52
 0.52  0.52  0.518 0.504 0.502 0.492 0.492 0.488 0.484 0.492 0.484 0.5
 0.49  0.49  0.498 0.502 0.496 0.502 0.506 0.514 0.52  0.514 0.524 0.522
 0.518 0.532 0.53  0.512 0.522 0.52  0.514 0.506 0.498 0.496 0.486 0.488
 0.482 0.478 0.494 0.514 0.5   0

Processing: infant3 segment3
file_index: 3
segment_index: 3
Loading ECG file:  infant3_ecg
Loading RESP file:  infant3_resp
Loading ECG file:  infant3_ecg
Loading RESP file:  infant3_resp
ECG sampling frequency:  500  Hz
RESP sampling frequency:  50  Hz

ECG stats:
Interval stats (n=400) :
min: 0.47 sec
max: 0.53 sec
mean: 0.50 sec (= 2.01 Hz)
std: 0.01 sec
q25: 0.49 sec
q75: 0.50 sec
[0.514 0.506 0.508 0.514 0.504 0.514 0.504 0.514 0.506 0.502 0.502 0.496
 0.506 0.508 0.51  0.51  0.514 0.526 0.51  0.5   0.498 0.5   0.494 0.498
 0.494 0.496 0.498 0.5   0.512 0.506 0.504 0.51  0.504 0.51  0.506 0.506
 0.506 0.498 0.5   0.496 0.498 0.492 0.49  0.498 0.494 0.5   0.498 0.5
 0.504 0.516 0.506 0.5   0.502 0.496 0.502 0.504 0.504 0.502 0.498 0.506
 0.498 0.502 0.496 0.502 0.506 0.5   0.512 0.502 0.51  0.502 0.508 0.5
 0.504 0.5   0.5   0.492 0.496 0.496 0.496 0.498 0.498 0.502 0.508 0.502
 0.504 0.512 0.508 0.512 0.506 0.502 0.498 0.49  0.494 0.49  0.494 0.492
 0.492 0.498 0.494 0.496 0.494 0



---DONE---
